# 画像データとテキストデータの両方を同時に扱うことができるMMBTを用いる。

In [8]:
## colabでこのノートを実行する時のみ使うセル
# from google.colab import drive
# drive.mount('/content/drive')

# !pip install transformers[ja]
# !pip install --quiet sentencepiece

In [11]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from scipy.special import softmax

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

from transformers import (
    AutoTokenizer, AutoModel, MMBTForClassification, MMBTConfig, AutoConfig,
    Trainer, TrainingArguments,
)
import transformers

from torchvision.io import read_image
from torchvision.models import ResNet152_Weights, resnet152

from matplotlib import pyplot as plt
import seaborn as sns